In [77]:
import pandas as pd
import numpy as np
import re

#opens file
file = open("sampleLOB.txt")

In [78]:
#stores data from file in array and strips whitespace
data = [line for line in file.readlines() if re.search("[\w].*", line) != None][1:]

In [79]:
organisedData = []
#converts to human readable lines
for line in data:
    for word in line.split(" "):
        if word != '\x00':
            charArray = word.split("\x00")[:-1]
            modifiedWord = ""
            for char in charArray:
                modifiedWord += char
                
            #removes speech marks and commas of time, bid and ask lines
            if "time" in modifiedWord or "bid" in modifiedWord or "ask" in modifiedWord:
                modifiedWord = modifiedWord[1:-2]
            organisedData.append(modifiedWord)

In [80]:
def chunkArrayByWord(ls,chunkWord):   
    chunkedData = []
    chunk = []
    for line in ls:
        if line==chunkWord:
            chunkedData.append(chunk)
            chunk = []
        else:
            chunk.append(line)
    return chunkedData

chunkedData = chunkArrayByWord(organisedData, 'time')[1:]

#remove entries where there are no entries in the order log book
chunkedData = list(filter(lambda timeEntry: len(timeEntry)>3, chunkedData))

evenChunkierData = []
for time in chunkedData:
    bidPos = 1 #it will always be 1
    askPos = time.index('ask')
    evenChunkierData.append([time[0], time[bidPos+1:askPos], time[askPos+1:]])

organisedData = []
for time in evenChunkierData:
    bids = []
    asks = []
    for i, bid in enumerate(time[1]):
        if i % 2 != 0:
            bids.append(time[1][i-1:i+1])
    for i, ask in enumerate(time[2]):
        if i % 2 != 0:
            asks.append(time[2][i-1:i+1])
    organisedData.append([time[0], bids, asks])

In [81]:
# now we aim to flatten the data into tabular data which we will eventually use for the csv file
tabularData =[]
for time in organisedData:
    for bids in time[1]:
        tabularData.append([
            float(time[0][:-1]), #removes comma and converts to float
            'bid',
            int(bids[0][:-1]), #removes comma and converts to int
            int(bids[1]) #converts to int
        ])
    for asks in time[2]:
        tabularData.append([
            float(time[0][:-1]), #removes comma and converts to float
            'ask',
            int(asks[0][:-1]),#removes comma and converts to int
            int(asks[1]) #converts to int
        ])       
np.array(tabularData)[:,3]

array(['3', '3', '3', ..., '4', '2', '5'], dtype='<U32')

In [82]:
#We can save this uncompressed data as csv data
dict = {
    'Time': np.array(tabularData)[:,0],
    'Transaction': np.array(tabularData)[:,1],
    'Value': np.array(tabularData)[:,2],
    'Amount': np.array(tabularData)[:,3],
}  
       
df = pd.DataFrame(dict)
df.to_csv('tabularData.csv') 

In [83]:
#we now need to the start and end time of a bid or ask
def createEntryWithStartAndEndTime(tabularData):
    tabularData = list(map(lambda row: [int(row[0]/0.016),row[0],row[1],row[2], row[3]], tabularData))
    initialEntryOfInterest = tabularData[0]
    startDate = tabularData[0][1]
    endDate = -1
    
    rowsOfInterest = list(filter(lambda e: e[-3:] == initialEntryOfInterest[-3:], tabularData))
    historyOfAnEntry = []
    #historyOfAnEntry = [initialEntryOfInterest]
    
    for i, row in enumerate(rowsOfInterest):
        if (i+1)==len(rowsOfInterest) or row[0] == rowsOfInterest[i+1][0]:
            historyOfAnEntry = historyOfAnEntry + rowsOfInterest[:i+1]
    if len(historyOfAnEntry) != 0:
        endDate = historyOfAnEntry[-1][1]
    else:
        historyOfAnEntry.append(initialEntryOfInterest)
        endDate = -1
    
    tabularData = list(map(lambda row: row[1:], filter(lambda row: row not in historyOfAnEntry, tabularData)))
    
    return [startDate, endDate] + initialEntryOfInterest[2:], tabularData

def compressDataToStartAndEndTime(tabularData):
    data = tabularData
    
    compressed = []
    while data != []:
        entry, data = createEntryWithStartAndEndTime(data)
        compressed.append(entry)
    
    return np.array(compressed)
    
compressed = compressDataToStartAndEndTime(tabularData)
print(compressed)

[['0.48' '2.592' 'bid' '27' '3']
 ['1.504' '3.104' 'bid' '183' '4']
 ['1.696' '2.992' 'ask' '739' '5']
 ['2.416' '5.2' 'ask' '222' '2']
 ['2.528' '3.12' 'bid' '172' '5']
 ['2.608' '2.912' 'bid' '139' '3']
 ['2.928' '3.856' 'bid' '95' '3']
 ['2.96' '4.992' 'bid' '196' '3']
 ['3.008' '5.2' 'ask' '651' '5']
 ['3.12' '5.2' 'bid' '187' '4']
 ['3.136' '5.2' 'bid' '175' '5']
 ['3.52' '4.352' 'bid' '51' '3']
 ['3.872' '4.24' 'bid' '49' '3']
 ['4.256' '5.2' 'bid' '199' '3']
 ['4.368' '4.848' 'bid' '44' '3']
 ['4.608' '5.2' 'ask' '209' '4']
 ['4.864' '5.2' 'bid' '153' '3']
 ['5.008' '5.056' 'bid' '199' '6']
 ['5.072' '5.2' 'bid' '201' '3']]


the error lies with the last value of a history of transaction

In [84]:
#Now we just need to save the data as a csv file
dict = {
    'Start_Time': np.array(compressed)[:,0],
    'End_Time': np.array(compressed)[:,1],
    'Transaction': np.array(compressed)[:,2],
    'Value': np.array(compressed)[:,3],
    'Amount': np.array(compressed)[:,4],
}  
       
df = pd.DataFrame(dict)
df.to_csv('compressedTabularData.csv') 